# Лабораторная работа №1

## Задание 1. 
###### Найти набор данных (датасет) для классификации удовлетворяющий следующим условиям: 
- более 10 000 строк
- более 20 столбцов
- разные типы в столбцах 
- обязательно наличие целевого признака (таргета).

Был выбран датасет: [Homicide Reports, 1980-2014](https://www.kaggle.com/datasets/murderaccountability/homicide-reports)


### About Dataset:
The Murder Accountability Project is the most complete database of homicides in the United States currently available. This dataset includes murders from the FBI's Supplementary Homicide Report from 1976 to the present and Freedom of Information Act data on more than 22,000 homicides that were not reported to the Justice Department. This dataset includes the age, race, sex, ethnicity of victims and perpetrators, in addition to the relationship between the victim and perpetrator and weapon used.

Проект "Убийственная отчетность" - это самая полная база данных об убийствах в США, существующая на сегодняшний день. В эту базу данных включены данные об убийствах из Дополнительного отчета об убийствах ФБР с 1976 года по настоящее время, а также данные, полученные в соответствии с Законом о свободе информации, о более чем 22 000 убийств, о которых не было сообщено в Министерство юстиции. В этом наборе данных указаны возраст, раса, пол, этническая принадлежность жертв и преступников, а также родственные связи между жертвой и преступником и использованное оружие.

### Датасет: 
- 638 454 записей
- 24 столбца 
- целевой признак - Crime Solved (Раскрытое преступление)

## Задание 2 . 
###### Провести классификацию найденного датасета, методом к- ближайших соседей. В формате Markdown писать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

### Загрузка Библиотек

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

### Загрузка данных и предварительная обработка
###### Загрузка данных

In [2]:
data = pd.read_csv('database.csv',low_memory=False)
# Удаление ненужных столбцов
data = data.drop(columns=["Record ID", "Record Source"])

###### Выбираем случайное подмножество данных, т.к. загруженный датасет содержит 638 454 записей которые будут обрабатываться слишком долго

In [3]:
subset_data = data.sample(n=11000, random_state=42)

subset_data.head() # Просмотр первых нескольких строк данных

,Agency Code,Agency Name,Agency Type,City,State,Year,Month,Incident,Crime Type,Crime Solved,...,Victim Race,Victim Ethnicity,Perpetrator Sex,Perpetrator Age,Perpetrator Race,Perpetrator Ethnicity,Relationship,Weapon,Victim Count,Perpetrator Count
484984,CA03612,Victorville,Municipal Police,San Bernardino,California,2005,July,1,Murder or Manslaughter,No,...,White,Hispanic,Unknown,0,Unknown,Unknown,Unknown,Handgun,1,0
608445,TX10100,Harris County,Sheriff,Harris,Texas,2013,May,3,Murder or Manslaughter,No,...,White,Not Hispanic,Unknown,0,Unknown,Unknown,Unknown,Firearm,0,0
417612,AR06200,St. Francis,Sheriff,St. Francis,Arkansas,2001,June,1,Murder or Manslaughter,Yes,...,Black,Not Hispanic,Male,18,Black,Not Hispanic,Acquaintance,Handgun,0,0
372692,AZ01003,Tucson,Municipal Police,Pima,Arizona,1998,March,6,Murder or Manslaughter,No,...,White,Not Hispanic,Unknown,0,Unknown,Unknown,Unknown,Handgun,0,0
402028,WA02400,Okanogan,Sheriff,Okanogan,Washington,1999,August,1,Murder or Manslaughter,Yes,...,White,Unknown,Male,20,White,Unknown,Girlfriend,Rifle,0,0


###### Выбор признаков и целевой переменной

In [4]:
X = subset_data[['Victim Age', 'Victim Race', 'Victim Sex', 'Perpetrator Age', 'Perpetrator Race', 'Perpetrator Sex']]
Y = subset_data['Crime Solved'] 

###### Преобразование категориальных признаков в числовые

In [5]:
X = pd.get_dummies(X)

###### Разделение данных на обучающий и тестовый наборы

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

###### Масштабирование данных
__*fit_transform*__ вычисляет среднее значение и стандартное отклонение для каждого признака на обучающем наборе данных

__*transform*__ применяет параметры масштабирования, вычисленные на обучающем наборе данных, к тестовому набору данных

In [7]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

###### Создание и обучение модели

Проведем перекрестную проверку для модели k-ближайших соседей с разными значениями параметра n_neighbors и выведем результаты для каждого значения:

In [8]:
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier

neighbors_list = [3, 5, 7, 9, 11]

for n_neighbors in neighbors_list:
    knn = KNeighborsClassifier(n_neighbors=n_neighbors)
    scores = cross_val_score(knn, X_train_scaled, Y_train, cv=5)
    print(f"n_neighbors = {n_neighbors}, Средняя оценка: {scores.mean()}")

n_neighbors = 3, Средняя оценка: 0.9954545454545455
n_neighbors = 5, Средняя оценка: 0.9928409090909092
n_neighbors = 7, Средняя оценка: 0.9910227272727272
n_neighbors = 9, Средняя оценка: 0.9902272727272727
n_neighbors = 11, Средняя оценка: 0.9894318181818182


In [9]:
knn = KNeighborsClassifier(n_neighbors=5)  # выберем 5 ближайших соседей
knn.fit(X_train_scaled, Y_train)

KNeighborsClassifier()

###### Прогнозирование на тестовом наборе данных

In [10]:
Y_pred = knn.predict(X_test_scaled)

###### Оценка точности модели

In [11]:
print( "Точность:", accuracy_score(Y_test, Y_pred) )

Точность: 0.9940909090909091


###### Вывод отчета о классификации

In [12]:
print( classification_report(Y_test, Y_pred) )

              precision    recall  f1-score   support

          No       0.98      1.00      0.99       650
         Yes       1.00      0.99      1.00      1550

    accuracy                           0.99      2200
   macro avg       0.99      1.00      0.99      2200
weighted avg       0.99      0.99      0.99      2200



###### Precision (точность): __TP / (TP + FP)__
Это показатель того, как много из предсказанных положительных случаев действительно являются положительными. 

###### Recall (полнота): __TP / (TP + FN)__
Это показатель того, как много реальных положительных случаев модель предсказала правильно.

###### F1-score: __2 * (precision * recall) / (precision + recall)__
Это гармоническое среднее между precision и recall. Это показатель общей производительности классификатора. 

###### Support: 
Это количество фактических вхождений каждого класса в тестовом наборе данных
___
###### Accuracy (точность) __(TP + TN) / (TP + TN + FP + FN)__ 
Это процент правильно классифицированных случаев. 

- TP - количество верно предсказанных положительных случаев 
- TN - количество верно предсказанных отрицательных случаев 
- FP - количество ложно предсказанных положительных случаев 
- FN - количество ложно предсказанных отрицательных случаев 

###### Macro avg: 
Средние значения __precision__, __recall__ и __f1-score__ по всем классам без учета их дисбаланса.

###### Weighted avg: 
Средние значения __precision__, __recall__ и __f1-score__ по всем классам, учитывая их дисбаланс по количеству.